In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
VOCAB, EMBEDDING_MATRIX = np.load("data/vocab_glove_100d.npy", allow_pickle=True).item(), np.load("data/embedding_matrix_glove_100d.npy", allow_pickle=True)

In [14]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch import nn
import pytorch_lightning as pl
import torchmetrics
import torch.nn.functional as F

class SWEMMultiNoPadClassifier(pl.LightningModule):
    def __init__(self, num_classes=3, embedding_dim=100, learning_rate=1e-3):
        super().__init__()
        self.linear = nn.Linear(embedding_dim, num_classes)
        self.model = nn.Sequential(self.linear)
        self.criterion = nn.BCEWithLogitsLoss()
        self.learning_rate = learning_rate
        self.train_accuracy = torchmetrics.Accuracy()
        self.val_accuracy = torchmetrics.Accuracy()
        self.train_precision = torchmetrics.Precision(num_classes=3, average='weighted')
        self.val_precision = torchmetrics.Precision(num_classes=3, average='weighted')
        self.train_recall = torchmetrics.Recall(num_classes=3, average='weighted')
        self.val_recall = torchmetrics.Recall(num_classes=3, average='weighted')
        self.train_f1 = torchmetrics.F1Score(num_classes=3, average='weighted')
        self.val_f1 = torchmetrics.F1Score(num_classes=3, average='weighted')
        self.save_hyperparameters()
    
    def forward(self, X):
        outputs = self.model(X)
        probs = F.sigmoid(outputs)
        return probs
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        outputs = self.model(X)
        train_loss = self.criterion(outputs, y.float())
        preds = self.forward(X)
        self.train_accuracy(preds, y)
        self.train_precision(preds, y)
        self.train_recall(preds, y)
        self.train_f1(preds, y)
        self.log("train_loss", train_loss, on_epoch=True)
        self.log('train_accuracy', self.train_accuracy, on_epoch=True)
        self.log('train_precision', self.train_precision, on_epoch=True)
        self.log('train_recall', self.train_recall, on_epoch=True)
        self.log('train_f1', self.train_f1, on_epoch=True)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        X, y = batch
        outputs = self.model(X)
        val_loss = self.criterion(outputs, y.float())
        preds = self.forward(X)
        self.val_accuracy(preds, y)
        self.val_precision(preds, y)
        self.val_recall(preds, y)
        self.val_f1(preds, y)
        self.log("val_loss", val_loss, on_epoch=True)
        self.log('val_accuracy', self.val_accuracy, on_epoch=True)
        self.log('val_precision', self.val_precision, on_epoch=True)
        self.log('val_recall', self.val_recall, on_epoch=True)
        self.log('val_f1', self.val_f1, on_epoch=True)
        return val_loss

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [4]:
# from relation_modeling_utils import load_data, HeadDataset

# train_df = load_data("data/atomic2020_data-feb2021/train.tsv", multi_label=True)
# dev_df = load_data("data/atomic2020_data-feb2021/dev.tsv", multi_label=True)
# train_data = HeadDataset(train_df, vocab=VOCAB, embedding_matrix=EMBEDDING_MATRIX, apply_pooling=True)
# val_data = HeadDataset(dev_df, vocab=VOCAB, embedding_matrix=EMBEDDING_MATRIX, apply_pooling=True)

In [5]:
# torch.save(train_data, "data/head_train_multi_no_pad.pt")
# torch.save(val_data, "data/head_val_multi_no_pad.pt")

In [15]:
train_data = torch.load("data/head_train_multi_no_pad.pt")
val_data = torch.load("data/head_val_multi_no_pad.pt")

In [16]:
train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=128)

In [17]:
from pytorch_lightning.loggers import WandbLogger
import wandb

wandb_logger = WandbLogger(project="kogito-relation-matcher", name="swem_multi_label_no_padding")
model = SWEMMultiNoPadClassifier(learning_rate=1e-4)
trainer = pl.Trainer(max_epochs=20, logger=wandb_logger)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
wandb.finish()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs



   | Name            | Type              | Params
-------------------------------------------------------
0  | linear          | Linear            | 303   
1  | model           | Sequential        | 303   
2  | criterion       | BCEWithLogitsLoss | 0     
3  | train_accuracy  | Accuracy          | 0     
4  | val_accuracy    | Accuracy          | 0     
5  | train_precision | Precision         | 0     
6  | val_precision   | Precision         | 0     
7  | train_recall    | Recall            | 0     
8  | val_recall      | Recall            | 0     
9  | train_f1        | F1Score           | 0     
10 | val_f1          | F1Score           | 0     
-------------------------------------------------------
303       Trainable params
0         Non-trainable params
303       Total params
0.001     Total estimated model params size (MB)


/Users/mismayil/opt/anaconda3/envs/kogito/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/mismayil/opt/anaconda3/envs/kogito/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/mismayil/opt/anaconda3/envs/kogito/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `Data

Epoch 19: 100%|██████████| 305/305 [00:04<00:00, 65.05it/s, loss=0.373, v_num=ar0u]



epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_accuracy_epoch,▁▅▅▆▆▇▇▇▇▇▇▇████████
train_accuracy_step,▁▂▄▆▅▅▆▆▆▆▆▇▇▆▇▇▆▆▆▆█▆█▇▆▇▇▇▇▆▇███▇▇▇▇▇▆
train_f1_epoch,▁▅▅▅▆▆▆▇▇▇▇▇▇███████
train_f1_step,▁▃▄▆▅▅▆▆▅▆▆▇▇▆▇▇▆▆▆▆█▆█▇▆▇▇▇▇▆▇▇██▇▇▇▇▇▆
train_loss_epoch,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train_loss_step,█▇▇▆▆▆▅▅▅▄▄▃▃▄▃▃▄▄▃▃▂▃▂▃▃▃▃▂▂▃▂▁▁▁▂▂▂▂▃▃
train_precision_epoch,▁▅▆▆▇▇▇▇▇▇▇█████████
train_precision_step,▁▁▃▅▅▅▆▆▅▆▆▇▇▆▇▆▆▆▆▆█▆█▇▆▆▆▇▇▆▇▇██▇▇▇▆▆▆
train_recall_epoch,▁▆▆▆▇▇▇▇▇▇▇▇████████
train_recall_step,▁▄▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▆▆▇█▇█▇▆▇▇▇▇▇▇▇██▇▇█▇▇▆
